# Source 1 : [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

## Recurrent Neural Networks (RNNs)

Traditional ANNs are unable to perceive the sequential context of data points, which is of course a fundamental ability in comprehending all sorts of things.

RNNs address this by including loops where data can persist. For input $x_t$, the network outputs some $h_t$. On the next input, $x_{t+1}$, the RNN receives persisting data from earlier inputs.

![RNN unrolled](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

RNNs, with this sequential architecture, have been successfully applied to numerous problems including speech recognition, translation, and image captioning. That said, LSTMs have been crucial for these successes.

## RNNs: Bad at Long-Term Dependencies

Based on what we've seen, RNNs should be able to connect previously seen information to their present task. As an example, say the network needs to predict the next word in a sentence based on previous words. If it's given "the clouds are in the ____," it's pretty obvious that the next word will be "sky." The immediate context enables the RNN to use the past information.

Consider a much longer paragraph beginning, "I grew up in France..." and ending with "I speak fluent ____." Recent context might inform the RNN that the next word will be a language. Unfortunately, the context of France is a much more distant memory.

As this gap grows, RNNs are unable to connect the distant information. In theory, they're capable of handling these "long-term dependencies," but in practice they're unable to learn the exact parameters to do so on their own. To quote the title of Yoshua Bengio's [1994 paper](http://ai.dinfo.unifi.it/paolo//ps/tnn-94-gradient.pdf) on the topic, "learning long-term dependencies with gradient descent is difficult."

## LSTMs: Good at Long-Term Dependencies

Long Short Term Memory networks - shortened to "LSTMs" - are a special kind of RNN capable of long-term dependencies. They were introduced in 1997 in [this paper](http://www.bioinf.jku.at/publications/older/2604.pdf). LSTMs were designed to solve the long-term dependency problem, and thus remembering information for a long time is basically their default behavior.

Consider the structure of a typical RNN:

![RNN Chain](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)

In the RNN, the chain of repeating modules is quite simple: the output of each layer is combined with the next input. We then $\text{tanh}$ the whole thing.

For clarity, consider another RNN illustration:

![RNN Alternative View](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

Notation from [another source](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/):
* $x_t$ is input at time $t$.
* $s_t$ is the hidden state at time $t$. It's the memory of the RNN and is calculated as $s_t=f(Ux_t+Ws_{t+1})$. $f$ is a nonlinearity like $\text{tanh}$ or $\text{ReLU}$.
* $o_t$ is the output at time $t$. As an example, it might be $\text{softmax}(Vs_t)$ if we wanted to select the highest probabilty of the options in the output vector.
* $U$, $V$, and $W$ are the same for each module, as can be seen on the left.

Now consider the structure of an LSTM:

![LSTM Chain](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

LSTMs instead have four neural network layers per module. The notation used is as follows:

![RNN Notation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM2-notation.png)

We'll go into the specifics of the LSTM structure later.

## The Core Idea of LSTMs

The centerpiece of the LSTM is the cell state, seen as the top line in each module. These run straight down the chain, with optional changes occasionally altering the information passed along. The few ways the LSTM can add or remove data from the cell state are called gates.

Gates optionally let information through. They're composed of a sigmoid layer and a pointwise multiplication operation. Sigmoid (by definition) outputs numbers zero to one, where a zero says "change nothing in the cell state" and a one meaning "let everything through!"

LSTMs have three of these gates:
1. Forget gate
2. Input gate
3. Output gate

## Step-by-step Walkthrough

Step 1: Decide what information in the cell state should be thrown away. This is made by the "forget gate layer." Looks at $h_{t-1}$ and $x_t$ and outputs a number 0-1 for each number in cell state $C_{t-1}$. 1 says to keep and 0 says to completely remove.  
![Forget Gate Graphic](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png)

Step 2: Decide what information we should store in the cell state. This has two subprocesses: a $\text{sigmoid}$ layer called the "input gate layer" decides which values to update, and a $\text{tanh}$ layer creates a vector of candidate values, $\tilde{C}_t$,  as potential options to add to the cell state.  
![Input Gate Layer Graphic](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png)

Step 3: Update the old cell state $C_{t-1}$ into new cell state $C_t$. We already have *what* to do based on the previous steps, but we now *do* it:  
1. Multiply $C_{t-1}$ by $f_t$, the forget gate's output. This "forgets" what we no longer need.
2. Add $i_t*\tilde{C}_t$ to this result. The new candidate values, scaled by their importance, were what we'd decided was worth remembering from this step.  
![Update Cell State Graphic](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png)

Step 4: Decide what we'd like to output. This will be a filtered version of our cell state:
1. We use a sigmoid layer over to determine which parts of the cell state to output.
2. We put the cell state through a $\text{tanh}$ layer to push the values between -1 and 1.
3. Multiply the sigmoid's result by the tanh scaling as to output only what we decided on.  
![Output Layers Graphic](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png)

## Variants on LSTMs

We've thus far described "a pretty normal LSTM," but in reality almost every paper using LSTMs uses a slightly different version.

**Peephole Connections** - Introduced in 2000 by [Gers & Schmidhuber](ftp://ftp.idsia.ch/pub/juergen/TimeCount-IJCNN2000.pdf). These additional connections allow the gate layers to look at the cell state. Many papers only give some gates peepholes and not others.  
![Peephole Connections Graphic](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-peepholes.png)

**Coupled Forget and Input Gates** - Instead of separately deciding where to forget or add new information, we combine those decisions. We only forget something if we're going to input something in its place.  
![Coupled Forget/Input Gates Graphic](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-tied.png)

**GRU** - Gated Recurrent Units, or GRUs, were introduced by [Cho, et al.](https://arxiv.org/pdf/1406.1078v3.pdf) in 2014. These modules combine the forget and input gates into a single "update gate." They also merge the cell and hidden states, among other changes. The result is simpler than standard LSTMs and was growing in popularity as of 2015.  
![GRU Graphic](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png)

**Additional Variants** - There are numerous other LSTM variants. Some of these include [Depth Gated RNNs](https://arxiv.org/pdf/1508.03790v2.pdf) or [Clockwork RNNs](https://arxiv.org/pdf/1402.3511v1.pdf). In general, most variants achieve pretty similar results, but of course [a few can do better than LSTMs](http://proceedings.mlr.press/v37/jozefowicz15.pdf) on certain tasks.

## Conclusion

LSTMs are responsible for many of the achievements being made with RNNs. The model may be complex, but hopefully it's a bit more approachable now. The next step, in many researchers' opinions, is the use of attention in ML/DL models.

# Source 2: [Beginner's Guide to LSTMs and RNNs](https://skymind.ai/wiki/lstm)

**Summary**  
The goal of this article is to give intuitions about how RNNs function, as well as the purpose and structure of LSTMs, a popular variation on RNNs.  
Recurrent networks are designed to recognize patterns in sequential data, such as text, genomes, handwriting, or time series data. They take time, or at least the sequence of data, into account. This gives them a temporal dimension.

## Review of Feedfoward Networks

Before getting into RNNs, let's establish a firm understanding of the more basic feedforward network. These are both named after the way they channel data: one *feeds data forward* through the network, where information never touches a node twice, while the other cycles data through a loop *recurrently*.

Feedforward Network:  
![Feedforward Network](https://www.researchgate.net/profile/Ramon_Quiza/publication/234055177/figure/fig1/AS:300092981563410@1448559150651/Sample-of-a-feed-forward-neural-network.png)

In a feedforward network, input examples are fed into the network and transformed into some output. In the case of supervised learning, this output might be a label for the input (classification), or a numerical estimate (regression). The network is trained to minimize its error, or how far off its guesses are from the correct outputs. Once trained, the network should be able to correctly classify new examples it's never seen before.

Furthermore, as the network classifies these hypothetical new examples, its prediction on one image won't impact its prediction on the next. Each example is judged independent of order or sequence, i.e. the network has no sense of time.

## Recurrent Networks

But there are some cases where the order of data matters. Recurrent neural networks, or RNNs, are one way to handle this need. RNNs consider their current input *as well as* what they've previously seen.

We can express this sense of time by considering each example as a given time step. We say the current decision is at time step $t$, and thus the previous decision was at time $t-1$. Thus any decisions a time $t$ consider data from $t-1$, or even earlier, depending on the RNN's architecture.

RNNs, unlike feedforward networks, receive their past decisions in that feedback loop. These loops give RNNs something like a memory, which is only useful given sequential data. In such cases, RNNs can perform tasks feedforward networks cannot.

The sequential information between time steps is preserved in the RNNs *hidden state*. This state spans many time steps, carrying important information forward through time. Thus RNNs can hypothetically detect patterns across long stretches of time. These are called "long-term dependencies," as an event much later in time might be the result of one or more previous events.  
One way to conceptualize it is that RNNs allow us to share weights over time.

## Formalizing RNNs

Mathematically, we express this memory as:  
$h_t=\phi(Wx_t+Uh_{t-1})$,  
where the hidden state at time $t$ is $h_t$. This value is a function of the input at the same time step, $x_t$, modified by some weight matrix $W$. This is added to the previous hidden state, $h_{t-1}$, multiplied by another weight matrix $U$. These matrices are also known as transition matrices, similar to Markov chains (**???**).

In an intuitive sense, these weight matrices allow us to determine the importance of both the present input and the past hidden state. We can use backpropagation to take the error at each time step and adjust the weights to minimize said error.

The sum of the two terms is then squashed into a specific range by the $\phi$ function. This can either by $sigmoid$ or $tanh$, **unsure why either would be preferable**.

Either way, because this feedback loop occurs at every time step, $h_t$ can contain data not only from $h_{t-1}$ but also all those that preceded $h_{t-1}$ as far back as the RNN's memory persists.

## Backpropagation Through Time (BPTT)

In order to make increase the accuracy of our predictions, we need to somehow change the $W$ and $U$ matrices. Our goal in doing this will be to give certain features of the input the importance they deserve in our predictions, and to store important features in the hidden state for future time steps.

As in a typical neural network, we can use backpropagation of error and gradient descent to improve these weight matrices. In a feedforward network, this process moves the final error backwards through the network, assigning each of the weights a "responsibility" for their portion of the error. This is done by calculating their partial derivatives, $\delta{E}/\delta{w}$, with respect to the error. We then adjust the weights accordingly.

Recurrent networks extend this backpropagation through time. Time here means the calculations between each step of the RNN, which is all backprop needs to work. All neural networks are nested composite functions, like $f(g(h(x)))$. The time element in RNNs only extends the series of functions for which we need to calculate the derivative. The chain rule doesn't mind.

**Truncated BPTT** - Truncated BPTT approximates full BPTT and is thus preferred for longer sequences. It's less computationally intensive, but its gradient can only flow so far back and it thus cannot learn quite as long dependencies as BPTT.

## Vanishing/Exploding Gradients

Recurrent nets are rather old, in fact so much so that even in the early 1990s the *vanishing gradient problem* had emerged as an issue with RNN performance. Recall that the gradient is what allows us to adjust the network's weights to decrease error. The issue comes from RNNs which need to recall extremely distant inputs: the many stages of multiplication from derivatives can make a gradient vanish or explode in just a few steps. The RNN's time steps exacerbate this problem even more so than in traditional networks.

Exploding gradients treat weights as more important than they are, saturating their meaning. The solution here is to truncate or squash the gradient, but vanishing gradients are more difficult to resolve. As you $\text{sigmoid}$ something over and over, all data is basically lost.

## LSTMs

In the mid-90s, Hochreiter and Schmidhuber proposed LSTMs as a solution to the vanishing gradient problem. LSTMs preserve the error across many time steps, thus allowing recurrent networks to continue across many more time steps. This makes LSTMs particularly useful, as they can confront environments with sparse or delayed signals.

LSTMs achieve this improvement by storing information outside the normal layer-to-layer flow of an RNN. They keep a gated cell where data can be stored, written in, read out, or erased. Each of these decisions is made by the LSTM based on what it learns over time. These decisions are analog, made with gates which enact element-wise multiplication using numbers from a $\text{sigmoid}$ and thus between 0 and 1.

These gates store their own weights to make these decisions, learned over time using the same recurrent backpropagation. Thus we have the benefits of differentiability while avoiding the vanishing gradient of long-term constant-access memory.

The article's diagram was truly awful, but here's a second one it provided:  
![LSTM Alternative Image](https://skymind.ai/images/wiki/greff_lstm_diagram.png)

It's important to note where the LSTM's memory cell uses multiplication versus addition. The central plus sign is the *magic secret* of LSTMs. By adding the new change to the cell state instead of multiplying, the derivative behaves much more nicely.

Different sets of weights filter the input for the input, forget, and output gates. The forget gate is effectively linear until a decision is made to erase information. It's been found that [adding a bias of 1](http://proceedings.mlr.press/v37/jozefowicz15.pdf) to the forget gate improves LSTM performance. The utility of a forget gate is that it allows the LSTM to clear between documents, for example, or perhaps songs.

It's important to note that RNNs can map one input to many (one image to many words in a caption), many to many (translation), or many to one (classifying a voice's gender).

## Capturing Diverse Time Scales and Remote Dependencies

One may wonder what the utility is of input gates protecting a memory cell from new data or output gates preventing it from affecting certain outputs. The utility is that these allow the RNN to work on different scales of time simultaneously.

CONTINUE

# Future Things To Read

On this: https://skymind.ai/wiki/lstm

Future:
https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714
https://towardsdatascience.com/understanding-lstm-and-its-quick-implementation-in-keras-for-sentiment-analysis-af410fd85b47
https://towardsdatascience.com/recurrent-neural-networks-and-lstm-4b601dd822a5
https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21  
Textbook: http://www.felixgers.de/papers/phd.pdf